Adapted from the notebook found at [How to Build a Law Bot](https://lawyerist.com/how-build-law-bot/)

## Install libraries

If you haven't already, you may need to install some dependencies. On the command line, run the following to install/update gspread, oauth2client, PyOpenSSL, and python-twitter.
```
pip install gspread
pip install --upgrade oauth2client
pip install PyOpenSSL
pip install python-twitter
```
Library installs are one and done. So after doing this once, you should be all set. 

## Import modules and set variables

Now we're getting into the bot's code. This is what will run every time your bot is called. 

You will need to create a new Google Sheet (same instructions as [last time](https://lawyerist.com/126074/online-forms-meet-local-document-automation-cut-and-paste-coding/)). Delete rows 2-999. This is because the code below appends values to the end of your sheet. So if you fail to remove rows 2-999, values will be appended to row 1000. Additionally, it looks at the last row of the sheet for your old values. So right off the bat it will be looking at your one solitary row. Also, delete columns D through Z to avoind having to print a bunch of empty columns.

As for a Twitter account and Twitter credentials, follow the instruction in [this post](https://lawyerist.com/?p=127093). 

In [1]:
# Load the module for visiting and reading websites.
import urllib.request
# Load the module for running regular expressions (regex).
import re 
# Load the module for date and time stuff.
import datetime
# Define the variable now as equal to the current date and time.
now = datetime.datetime.now()

In [3]:
# Set the URL you want to scrape.
url_1 = "http://twitter.com/realDonaldTrump?ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor"

# If you want to scrape data from multiple pages, you can, 
# just replicate the above and below but change url_1 to url_2 et al.

In [8]:
# Load the module for accessing Google Sheets.
import gspread
# Load the module needed for securely communicating with Google Sheets.
from oauth2client.service_account import ServiceAccountCredentials
# The scope for your access credentials
scope = ['https://spreadsheets.google.com/feeds']

# Your spreadsheet's ID
document_key = "13bsf7rP9ubfBSX7-hK3QayZIaUkTxrXaOZqGsfLjWX4" 
#              ^^^^^^^^^^^ SWAP OUT FOR YOUR DOCUMENT ID/KEY
# Your Google project's .json key
credentials = ServiceAccountCredentials.from_json_keyfile_name('../../../../../SheetsBot-32718db5596a.json', scope)
#                                                                              ^^^^^^^^ SWAP OUT FOR YOUR JSON KEY
# Use your credentials to authorize yourself.
gc = gspread.authorize(credentials)
# Open up the Sheet with the defined ID.
wks = gc.open_by_key(document_key)

#########################################
#
#  NOTE: The name of the sheet you are 
#  trying to access should be in the 
#  parenthetical below (e.g., Data). By
#  Default this is probably "Sheet1".
#
#########################################
worksheet = wks.worksheet("Sheet1")

# Count the number of rows in your Sheet &
# resize to remove blank rows.
worksheet.resize(worksheet.row_count)

FileNotFoundError: [Errno 2] No such file or directory: '../../../../../SheetsBot-32718db5596a.json'

In [ ]:
# Print out the old values stored in your sheet 
# Note: The first time you run this code, it will be empty as nothing has yet to be stored in your sheet.

print(worksheet.row_values(worksheet.row_count))
#############################
# DELETE CELL AFTER TESTING
#############################

In [ ]:
# Import the relevant Twitter libraries so you can use Twitter.
import twitter
from twitter import TwitterError

# create the following four text files and add them to the same diretctry as you 
# Google API key. In each file add the appropriate value found when retrieving your 
# Twitter credentials

with open('../../../../../key.txt', 'r') as myfile:
    key=myfile.read()
    
with open('../../../../../secret.txt', 'r') as myfile:
    secret=myfile.read()
    
with open('../../../../../token_key.txt', 'r') as myfile:
    token_key=myfile.read()

with open('../../../../../token_secret.txt', 'r') as myfile:
    token_secret=myfile.read()

# Set you Twitter API credentials.
api = twitter.Api(consumer_key=key,
                  consumer_secret=secret,
                  access_token_key=token_key,
                  access_token_secret=token_secret)

## Read the contents of your first webpage

When you run the next cell, your program will visit the first URL you defined above. It will then print out that page's HTML. 

In [5]:
p_1 = urllib.request.build_opener(urllib.request.HTTPCookieProcessor).open(url_1).read()
print(p_1)

b'<!DOCTYPE html>\n<html lang="en" data-scribe-reduced-action-queue="true">\n  <head>\n    \n    \n    \n    \n    \n    \n    \n    <meta charset="utf-8">\n    \n    <noscript><meta http-equiv="refresh" content="0; URL=https://mobile.twitter.com/i/nojs_router?path=%2FrealDonaldTrump&amp;ref_src=twsrc%5Egoogle%7Ctwcamp%5Eserp%7Ctwgr%5Eauthor"></noscript>\n      <script  nonce="S0k7TshPi7qB1tZnyA8ZqQ==">\n        !function(){window.initErrorstack||(window.initErrorstack=[]),window.onerror=function(r,i,n,o,t){r.indexOf("Script error.")>-1||window.initErrorstack.push({errorMsg:r,url:i,lineNumber:n,column:o,errorObj:t})}}();\n      </script>\n    \n    \n  \n  <script id="bouncer_terminate_iframe" nonce="S0k7TshPi7qB1tZnyA8ZqQ==">\n    if (window.top != window) {\n  window.top.postMessage({\'bouncer\': true, \'event\': \'complete\'}, \'*\');\n}\n  </script>\n  <script id="swift_action_queue" nonce="S0k7TshPi7qB1tZnyA8ZqQ==">\n    !function(){function e(e){if(e||(e=window.event),!e)return!1

------------------------------------

# One Data Point, One Match

## Parse the site's contents 

Scan the above HTML for the content you are trying to extract. Cut and paste the HTML above into the TEST STRING box over at [Regex 101](https://regex101.com/) and craft a regex that captures your desired content. Be sure to use the Python flavor.

Remember the parenthetical is the group you're pulling out. Once you have a working regex, plug it into the code below, and run the cell. If it worked, you'll see you scraped data as an output. 

In [6]:
res_1 = re.search(b"<title>(.*)</title>",p_1)
output_1 = res_1.group(1).decode('UTF-8')
print(output_1)

Donald J. Trump (@realDonaldTrump) | Twitter


## Post to Twitter and Save to Google

In [7]:
if (res_1 and (worksheet.row_values(worksheet.row_count)[1]) != output_1):
    # The above If statment, says to continue only if the we actuall got some data from the page
    # and the old sheet vales and the new pulled values are not equal (!=) to eachother. 

    # Go ahead and tweet out the update. Here you need to know about a Twitter API limitation.
    # Twitter will not Tweet the same tweet a second time if it is too close to the first instance.
    # In such cases, it will throw an error. The `try:` and `except TwitterError:` constructions are
    # similar to If statements. However, they will try the first block of code first, and only try 
    # the second block if it runs into a Twitter error. Here, the second try tweaks the language
    # just enough that it isn't a duplicate Tweet.
    try:
        # Post to Twitter.
        #print ('P1 Title is %s'%(output_1))
        status = api.PostUpdate('P1 Title is %s'%(output_1))
        print(status.text)
    except TwitterError:
        # Post to Twitter.
        #print ('P1 Title: %s'%(output_1))
        status = api.PostUpdate('P1 Title: %s'%(output_1))
        print(status.text)

    # Save to Google only after Tweeting
    worksheet.append_row([now,output_1])

NameError: name 'worksheet' is not defined

In [ ]:
print(worksheet.row_values(worksheet.row_count))
#############################
# DELETE CELL AFTER TESTING
#############################

# Two Data Points, One Match

---------------------------
## Parse the site's contents

In [ ]:
res_1 = re.search(b"<a href=\"([^\"]*)\"[^>]*>([^<]*)</a>",p_1)
output_1 = res_1.group(1).decode('UTF-8')
print(output_1)
output_2 = res_1.group(2).decode('UTF-8')
print(output_2)

## Post to Twitter and Save to Google (Two Data Point, One Match)

In [ ]:
if (res_1 and (worksheet.row_values(worksheet.row_count)[1]) != output_1
          and (worksheet.row_values(worksheet.row_count)[2]) != output_2):
    # same as above but now comparing two values
    
    try:
        # Post to Twitter.
        status = api.PostUpdate('%s %s'%(output_2,output_1))
        print(status.text)
    except TwitterError:
        # Post to Twitter.
        status = api.PostUpdate('%s %s'%(output_2,output_1))
        print(status.text)

    # Save to Google only after Tweeting
    worksheet.append_row([now,output_1,output_2])

In [ ]:
print(worksheet.row_values(worksheet.row_count))
#############################
# DELETE CELL AFTER TESTING
#############################

# Two Data Points, Multiple Fixed Number of Matches

-------------------------------------------

## Parse the site's contents and post to Twitter 

In [ ]:
matches = re.finditer(b"<a href=\"([^\"]*)\"[^>]*>([^<]*)</a>",p_1)
column = 0
columns = [now]

for matchNum, match in enumerate(matches):
    matchNum = matchNum + 1
    column = column + 1
    column2 = column + 1
    
    output_1 = match.group(1).decode('UTF-8')
    output_2 = match.group(2).decode('UTF-8')
    
    if (res_1 and (worksheet.row_values(worksheet.row_count)[column]) != output_1 
              and (worksheet.row_values(worksheet.row_count)[column2]) != output_2):
        # same as above but now comparing the two values handled in this loop
        
        try:
            # Post to Twitter.
            status = api.PostUpdate('%s %s'%(output_2,output_1))
            print(status.text)
        except TwitterError:
            # Post to Twitter.
            status = api.PostUpdate('%s %s'%(output_2,output_1))
            print(status.text)
            
        columns.append(output_1)
        columns.append(output_2)
        
    column = column + 1

# Save to Google only after Tweeting
worksheet.append_row(columns)

In [ ]:
print(worksheet.row_values(worksheet.row_count))
#############################
# DELETE CELL AFTER TESTING
#############################